In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import haversine_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Расстояние Хаверсайна

# Находим сходство между объектами из разных таблиц по координатам.

In [ ]:
# Функция для вычисления расстояния между двумя точками по координатам
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

# Загрузка данных
table1 = pd.read_csv('table1.csv')  # Замените на ваш путь к файлу
table2 = pd.read_csv('table2.csv')  # Замените на ваш путь к файлу

# Создаем новый столбец для хранения индексов схожих объектов
table1['similar_indices'] = [[] for _ in range(len(table1))]

# Определяем порог расстояния (в метрах)
distance_threshold = 500

# Вычисляем расстояния и заполняем столбец
for i, row1 in table1.iterrows():
    for j, row2 in table2.iterrows():
        distance = calculate_distance(row1['lat'], row1['lon'], row2['lat'], row2['lon'])
        if distance < distance_threshold:
            table1.at[i, 'similar_indices'].append(j)

# Сохранение результата в новый CSV файл
table1.to_csv('table1_with_similar.csv', index = False)

# Визуализация
def plot_similar_objects(row1, table2):
    plt.figure(figsize = (10, 6))
    
    # Плотность объектов из таблицы 2
    plt.scatter(table2['lon'], table2['lat'], color = 'blue', label = 'Элитное жилье', alpha = 0.5)

    # Объект из таблицы 1
    plt.scatter(row1['lon'], row1['lat'], color = 'red', label = 'Объект из таблицы 1', s = 100)

    # Схожие объекты
    for index in row1['similar_indices']:
        plt.scatter(table2.iloc[index]['lon'], table2.iloc[index]['lat'], color = 'green', label = 'Схожий объект', alpha = 0.7)

    plt.title('Схожие объекты')
    plt.xlabel('Долгота')
    plt.ylabel('Широта')
    plt.legend()
    plt.grid()
    plt.show()

# Визуализируем для первого объекта из таблицы 1
plot_similar_objects(table1.iloc[0], table2)

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Загрузка данных (замените 'table1.csv' и 'table2.csv' на ваши файлы)
df1 = pd.read_csv('table1.csv', encoding='utf-8')
df2 = pd.read_csv('table2.csv', encoding='utf-8')

# Предполагаем, что столбцы с координатами называются 'latitude' и 'longitude'
df1['coordinates'] = list(zip(df1['latitude'], df1['longitude']))
df2['coordinates'] = list(zip(df2['latitude'], df2['longitude']))


def find_matches(row, df2, max_distance_km):
    """Находит совпадения в df2 для данной строки из df1."""
    matches = []
    for index, row2 in df2.iterrows():
        distance = geodesic(row['coordinates'], row2['coordinates']).km
        if distance <= max_distance_km:
            matches.append(row2)
    return matches

# Задайте максимальное расстояние в километрах (настройте по необходимости)
max_distance_km = 0.1  # Например, 100 метров

# Применяем функцию к каждой строке df1
df1['matches'] = df1.apply(lambda row: find_matches(row, df2, max_distance_km), axis=1)

#  Вывод результатов.  Обратите внимание, что 'matches' - это список DataFrame
print(df1)

#Если нужно получить  более удобный для анализа формат, например,  создать столбец с ID совпавших квартир или другими данными:
#Пример: добавим ID из df2 к результатам
df1['matched_ids'] = df1['matches'].apply(lambda x: [row['id'] for row in x] if len(x)>0 else []) #id - предполагаемое имя столбца с ID в df2

print(df1)

1. geopy.distance.geodesic:  Эта функция вычисляет географическое расстояние между двумя точками, используя алгоритм Vincenty.  Это гораздо точнее, чем косинусное сходство для географических данных.

2. max_distance_km:  Этот параметр задает радиус поиска.  Экспериментируйте с его значением, чтобы найти оптимальное значение для вашего набора данных.  Начните с небольшого значения (например, 0.1 км или 100 метров), если вы ищете квартиры в одном и том же доме.

3. find_matches: Эта функция итерируется по df2 и находит все квартиры, расстояние до которых меньше max_distance_km.

4. Обработка результатов:  Результирующий df1 будет содержать столбец matches, который содержит списки DataFrame с совпадениями из df2.  Вы можете дальше обработать эти списки для извлечения необходимых данных.

In [ ]:
# Вычисляем матрицу косинусного сходства
cosine_similarities = cosine_similarity(df1['features'], df2['features'])

# еще обнулить диагональ!!!
# Устанавливаем порог для отбора: чем выше порог, тем строже соответствие
threshold = 0.999  # Настройте по необходимости

# Получаем индексы совпадений для каждой строки df1
matches = []
for i in range(len(df1)):
    matches.append([j for j in range(len(df2)) if cosine_similarities[i][j] >= threshold])

# Добавляем список совпадений в df1
df1['matches'] = matches

# Нормализация матрицы сходства 

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ... (загрузка данных, вычисление cosine_similarities, как в предыдущих примерах) ...


def normalize_similarity_matrix(similarity_matrix, method='row'):
    """Нормализует матрицу сходства."""
    if method == 'row':
        row_sums = np.sum(similarity_matrix, axis=1, keepdims=True)
        normalized_matrix = similarity_matrix / row_sums
        normalized_matrix = np.nan_to_num(normalized_matrix) # Обработка NaN, которые могут возникнуть при нулевой сумме строки.
        return normalized_matrix
    elif method == 'column':
        col_sums = np.sum(similarity_matrix, axis=0, keepdims=True)
        normalized_matrix = similarity_matrix / col_sums
        normalized_matrix = np.nan_to_num(normalized_matrix) # Обработка NaN
        return normalized_matrix
    elif method == 'global':
        global_sum = np.sum(similarity_matrix)
        normalized_matrix = similarity_matrix / global_sum
        return normalized_matrix
    else:
        raise ValueError("Invalid normalization method.")


# Нормализация по строке
normalized_cosine_similarities_row = normalize_similarity_matrix(cosine_similarities, method = 'row')

# Нормализация по столбцу
normalized_cosine_similarities_col = normalize_similarity_matrix(cosine_similarities, method = 'column')

# Глобальная нормализация
normalized_cosine_similarities_global = normalize_similarity_matrix(cosine_similarities, method = 'global')


# Вывод для сравнения:
print("Оригинальная матрица:")
print(cosine_similarities)
print("\nНормализованная по строкам матрица:")
print(normalized_cosine_similarities_row)
print("\nНормализованная по столбцам матрица:")
print(normalized_cosine_similarities_col)
print("\nГлобально нормализованная матрица:")
print(normalized_cosine_similarities_global)

# ... (Дальнейшая обработка с выбором порога, как описано в предыдущих ответах, но теперь
#      используя  normalized_cosine_similarities_row,  normalized_cosine_similarities_col
#      или normalized_cosine_similarities_global вместо cosine_similarities ) ...

Выбор метода нормализации:

• Нормализация по строке —  подходит, если вас интересует относительное сходство квартир из df1 с квартирами из df2.

• Нормализация по столбцу — подходит, если вас интересует, насколько каждая квартира из df2 похожа на квартиры из df1.

• Глобальная нормализация — подходит, если вы хотите получить относительное сходство между всеми парами квартир.

# Находим расстояние между объектами из разных таблиц (Квартира - Метро).

In [ ]:
# Функция для вычисления расстояния по формуле Хаверсина
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    lat1_rad = np.radians(lat1) #широта квартиры
    lon1_rad = np.radians(lon1) #долгота квартиры
    lat2_rad = np.radians(lat2) #широта метро
    lon2_rad = np.radians(lon2) #долгота метро

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c  # Расстояние в километрах
    return distance

# Загрузка данных
apartments = pd.read_csv('apartments.csv')  # Замените на ваш путь к файлу с квартирами
metro_stations = pd.read_csv('metro_stations.csv')  # Замените на ваш путь к файлу со станциями метро

# Создаем новые столбцы для ближайшей станции и расстояния
apartments['nearest_metro'] = None
apartments['distance_to_nearest_metro'] = None

# Вычисляем расстояния
for i, apartment in tqdm(apartments.iterrows(), total = apartments.shape[0], desc = 'Обработка квартир'):
    min_distance = float('inf')
    nearest_station = None
    
    for j, station in tqdm(metro_stations.iterrows(), total = metro_stations.shape[0], desc = 'Поиск ближайшей станции метро', leave = False):
        distance = haversine(apartment['lat'], apartment['lon'], station['lat'], station['lon'])
        
        if distance < min_distance:
            min_distance = distance
            nearest_station = station['name']  # Предполагается, что в таблице метро есть столбец 'name'
    
    apartments.at[i, 'nearest_metro'] = nearest_station
    apartments.at[i, 'distance_to_nearest_metro'] = min_distance

# Сохранение результата в новый CSV файл
apartments.to_csv('apartments_with_nearest_metro.csv', index = False)

print("Расчеты завершены. Результаты сохранены в 'apartments_with_nearest_metro.csv'.")

# Евклидово расстояние

In [ ]:
# Пример данных о квартирах (замените на ваши данные)
apartments = pd.DataFrame({
                           'apartment_id': range(1, 6),
                           'lat': [55.75, 55.76, 55.77, 55.78, 55.79],
                           'lon': [37.62, 37.63, 37.64, 37.65, 37.66]
                         })

# Пример данных о станциях метро (замените на ваши данные)
stations = pd.DataFrame({
                         'station_name': ['Station A', 'Station B', 'Station C'],
                         'lat': [55.755, 55.775, 55.795],
                         'lon': [37.625, 37.645, 37.665]
                       })


avg_lat_degree_meters = 111000

def calculate_approx_distance(lat1, lon1, lat2, lon2):
  
    avg_lat_degree_meters = 111320
    lat_diff_meters = abs(lat1 - lat2) * avg_lat_degree_meters

    # Рассчитываем длину градуса долготы на основе средней широты
    avg_latitude = (lat1 + lat2) / 2
    avg_latitude_radians = math.radians(avg_latitude)
    lon_degree_meters = 111320 * math.cos(avg_latitude_radians) # Более точный расчет
    lon_diff_meters = abs(lon1 - lon2) * lon_degree_meters

    return np.sqrt(lat_diff_meters**2 + lon_diff_meters**2)


def find_closest_station(row, stations):
    min_distance = float('inf')
    closest_station = None
    for index, station in stations.iterrows():
        distance = calculate_euclidean_distance(row['lat'], row['lon'], station['lat'], station['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_station = station['station_name']
    return closest_station, min_distance


apartments[['closest_station', 'distance']] = apartments.apply(lambda row: find_closest_station(row, stations), axis = 1).tolist()

print(apartments)

# Сходство объектов (сравнение векторов)

In [ ]:
# Загрузка данных (замените на ваши данные)
data = pd.DataFrame({
    'area': [50, 60, 70, 55, 65, 75, 52, 62, 72, 57],
    'lon': [37.6, 37.7, 37.6, 37.7, 37.6, 37.7, 37.6, 37.7, 37.6, 37.7],
    'lat': [55.7, 55.8, 55.7, 55.8, 55.7, 55.8, 55.7, 55.8, 55.7, 55.8],
    'floor': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1],
    'floors_total': [5, 5, 5, 10, 10, 10, 5, 5, 5, 10],
    'year': [2000, 2005, 2010, 2000, 2005, 2010, 2000, 2005, 2010, 2000],
    'district': ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B'],
    'distance_to_center': [5, 10, 15, 5, 10, 15, 5, 10, 15, 5],
    'terrace': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1], # 0 - нет террасы, 1 - есть терраса
    'price': [100000, 120000, 140000, 110000, 130000, 150000, 90000, 110000, 130000, 100000]
})


# Предобработка данных
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)

numerical_cols = ['area', 'lon', 'lat', 'floor', 'floors_total', 'year', 'distance_to_center']
categorical_cols = ['district']

data_numerical_scaled = scaler.fit_transform(data[numerical_cols])
data_categorical_encoded = encoder.fit_transform(data[categorical_cols])

data_processed = np.concatenate((data_numerical_scaled, data_categorical_encoded, data[['terrace']]), axis = 1)

# Задаем веса признаков
#weights = np.array([0.15, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2]) # пример весов

# Задаем порог схожести
similarity_threshold = 0.9 

# Вычисление косинусной схожести с помощью векторизованных вычислений
cosine_similarities = cosine_similarity(data_processed)
# Обнуление главной диагонали матрицы схожести
np.fill_diagonal(cosine_similarities, 0)

# K-Means
# kmeans = KMeans(n_clusters = 5)
# kmeans.fit(similarity_matrix)
# labels = kmeans.labels_

# Вычисление взвешенного евклидова расстояния
# distances = []
# for i in range(len(data_processed)):
#     row_distances = []
#     for j in range(len(data_processed)):
#         weighted_distance = np.sqrt(np.sum(weights * (data_processed[i] - data_processed[j])**2))
#         row_distances.append(weighted_distance)
#     distances.append(row_distances)
# distances = np.array(distances)

# Фильтрация по порогу
similar_objects = []
for i, row in enumerate(cosine_similarities): #distances
    similar_indices = np.where(row <= similarity_threshold)[0].tolist()
    similar_objects.append(similar_indices)

data['similar_objects'] = similar_objects

# Группировка по наличию террасы и анализ цен
group_no_terrace = data[data['terrace'] == 0]
group_terrace = data[data['terrace'] == 1]

avg_price_no_terrace = group_no_terrace['price'].mean() / group_no_terrace['area'].mean()
avg_price_terrace = group_terrace['price'].mean() / group_terrace['area'].mean()

price_difference_percentage = ((avg_price_terrace - avg_price_no_terrace) / avg_price_no_terrace) * 100

print(f"Средняя удельная цена без террасы: {avg_price_no_terrace:.2f}")
print(f"Средняя удельная цена с террасой: {avg_price_terrace:.2f}")
print(f"Процентная разница в цене: {price_difference_percentage:.2f}%")

#Визуализация
plt.figure(figsize = (10,6))
plt.scatter(data[data['terrace'] == 0]['area'], data[data['terrace'] == 0]['price'], label='Без террасы')
plt.scatter(data[data['terrace'] == 1]['area'], data[data['terrace'] == 1]['price'], label='С террасой')
plt.xlabel('Площадь')
plt.ylabel('Цена')
plt.legend()
plt.title('Влияние наличия террасы на цену')
plt.show()

plt.figure(figsize=(10,6))
plt.boxplot([group_no_terrace['price'], group_terrace['price']], labels=['Без террасы', 'С террасой'])
plt.ylabel('Цена')
plt.title('Сравнение цен с помощью box plot')
plt.show()

In [ ]:
# Выбор первого объекта и его похожих объектов
first_object_index = 0
similar_indices = data['similar_objects'][first_object_index]

# Создание новой таблицы с похожими объектами
similar_objects_df = data.iloc[similar_indices].copy()

# Вывод информации
print("Первый объект:")
print(data.iloc[first_object_index])
print("\nПохожие объекты:")
print(similar_objects_df)

# NearestNeighbors

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Пример данных (замените на ваши данные)
data = {
    'площадь': [50, 60, 70, 55, 65, 75, 52, 62],
    'этаж': [3, 5, 2, 4, 1, 6, 2, 7],
    'год_постройки': [2010, 2015, 2005, 2012, 2008, 2018, 2009, 2016],
    'район': [1, 2, 1, 3, 2, 1, 3, 2],
    'широта': [55.75, 55.76, 55.74, 55.755, 55.765, 55.745, 55.77, 55.758],
    'долгота': [37.62, 37.63, 37.61, 37.625, 37.635, 37.615, 37.64, 37.628]
}
df = pd.DataFrame(data)

# Нормализация данных (важно для большинства методов поиска ближайших соседей)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# k-NN с Ball Tree
k = 3  # Количество ближайших соседей
knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
knn.fit(scaled_data)

# Поиск похожих квартир (например, для первой квартиры)
distances, indices = knn.kneighbors(scaled_data[0].reshape(1, -1))

print("Показатели для первой квартиры:", df.iloc[0])
print("Индексы ближайших соседей:", indices[0])
print("Показатели ближайших соседей:")
print(df.iloc[indices[0]])
print("Расстояния до ближайших соседей:", distances[0])

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cityblock #Для Manhattan distance

# Пример данных (замените на ваши данные)
data = {
    'площадь': [50, 60, 70, 55, 65, 75, 52, 62, 80, 45],
    'этаж': [3, 5, 2, 4, 1, 6, 2, 7, 10, 1],
    'год_постройки': [2010, 2015, 2005, 2012, 2008, 2018, 2009, 2016, 2020, 2000],
    'район': [1, 2, 1, 3, 2, 1, 3, 2, 1, 4],
    'широта': [55.75, 55.76, 55.74, 55.755, 55.765, 55.745, 55.77, 55.758, 55.752, 55.78],
    'долгота': [37.62, 37.63, 37.61, 37.625, 37.635, 37.615, 37.64, 37.628, 37.618, 37.65]
}
df = pd.DataFrame(data)

# Нормализация данных
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# k-NN с KD-деревом и Manhattan distance
k = 3  # Количество ближайших соседей
knn = NearestNeighbors(n_neighbors=k, algorithm='kd_tree', metric='manhattan') # metric='manhattan' - ключевое изменение
knn.fit(scaled_data)

# Поиск похожих квартир (например, для первой квартиры)
distances, indices = knn.kneighbors(scaled_data[0].reshape(1, -1))

print("Показатели для первой квартиры:", df.iloc[0])
print("Индексы ближайших соседей:", indices[0])
print("Показатели ближайших соседей:")
print(df.iloc[indices[0]])
print("Расстояния до ближайших соседей (Manhattan):", distances[0])


#Пример использования cityblock для сравнения одной пары:

квартира1 = scaled_data[0]
квартира2 = scaled_data[1]

расстояние_manhattan = cityblock(квартира1, квартира2)
print(f"Расстояние Manhattan между квартирой 1 и 2: {расстояние_manhattan}")

cityblock:  Функция cityblock из библиотеки scipy.spatial.distance напрямую вычисляет расстояние Manhattan между двумя векторами.  Этот пример показывает, как её использовать, если вам нужно рассчитать расстояние между отдельными парами квартир.

Обратите внимание, что kd_tree эффективен для низкоразмерных данных.  Для высокоразмерных данных лучше использовать ball_tree или другие структуры данных, предназначенные для больших размерностей.  Если у вас много признаков,  возможно, стоит рассмотреть методы понижения размерности (PCA, t-SNE)  перед поиском ближайших соседей.  Это может улучшить как производительность, так и качество результатов.  Кроме того,  выбор между manhattan и euclidean зависит от вашей задачи и  характера данных.  Эксперименты на ваших данных помогут определить, какой метод работает лучше.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

# Пример данных (замените на ваши данные)
data1 = {'latitude': [55.75, 55.76, 55.77], 'longitude': [37.62, 37.63, 37.64]}
data2 = {'latitude': [55.755, 55.765, 55.775, 55.8, 55.7], 'longitude': [37.625, 37.635, 37.645, 37.7, 37.5]}

таблица_1 = pd.DataFrame(data1)
таблица_2 = pd.DataFrame(data2)


# 1. Объединение таблиц для стандартизации
combined_data = pd.concat([таблица_1, таблица_2])

# 2. Стандартизация данных с помощью MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

# 3. Разделение стандартизированных данных обратно на таблицы
таблица_1_scaled = scaled_data[:len(таблица_1),:]
таблица_2_scaled = scaled_data[len(таблица_1):,:]

# 4. Подготовка данных для NearestNeighbors
X1 = таблица_1_scaled # Данные из таблицы_1
X2 = таблица_2_scaled # Данные из таблицы_2

# 5. Создание и обучение модели NearestNeighbors
knn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric='manhattan') # Можно попробовать kd_tree или brute для сравнения
knn.fit(X2)

# 6. Поиск ближайших соседей
distances, indices = knn.kneighbors(X1)

# 7. Добавление индексов ближайших соседей в таблицу_2
таблица_2['nearest_neighbor_index'] = indices.flatten()

# 8. Вывод результатов
print("Таблица 1 (оригинальные данные):\n", таблица_1)
print("\nТаблица 2 (оригинальные данные) с индексами ближайших соседей:\n", таблица_2)
print("\nТаблица 1 (масштабированные данные):\n", pd.DataFrame(таблица_1_scaled, columns=['latitude', 'longitude']))
print("\nТаблица 2 (масштабированные данные):\n", pd.DataFrame(таблица_2_scaled, columns=['latitude', 'longitude']))

Можно сделать 3 шага:
1. Находим ближайшего соседа NearestNeighbors.
2. Рассчитываем расстояние между координатами ОО и ближайшего соседа.
3. Сравниваем адреса ОО и ближайшего соседа.